In [1]:
#!pip install ultralytics

In [9]:
import ultralytics
ultralytics.__version__

'8.1.10'

In [10]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov8s.pt')

In [41]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [42]:
tracker=Tracker()
count=0

In [43]:
cap=cv2.VideoCapture('traffic_india.mp4')

In [44]:
down={}
up={}

counter_down=[]
counter_up=[]

In [45]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can also use 'XVID'
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (1020, 500))  # 20.0 is the frame rate, (1020, 500) is the frame size

In [46]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    # Assuming 'model.predict' and 'tracker.update' are valid methods for your model and tracker
    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()  # Added this line
    px = pd.DataFrame(a).astype("float")

    list_bboxes = []
    for index, row in px.iterrows():
        x1, y1, x2, y2, _, d = row.astype(int)
        c = class_list[d]
        if c in ['car','bicycle', 'car', 'motorcycle', 'truck']:
            list_bboxes.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list_bboxes)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = (x3 + x4) // 2
        cy = (y3 + y4) // 2

        red_line_y = 198
        blue_line_y = 268
        offset = 5

        # Red line condition
        if red_line_y - offset < cy < red_line_y + offset:
            down[id] = cy
        if id in down:
            if blue_line_y - offset < cy < blue_line_y + offset:
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                counter_down.append(id)

        # Blue line condition
        if blue_line_y - offset < cy < blue_line_y + offset:
            up[id] = cy
        if id in up:
            if red_line_y - offset < cy < red_line_y + offset:
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                counter_up.append(id)

    # Drawing lines and texts on frame
    cv2.line(frame, (172, 198), (774, 198), (0, 0, 255), 3)
    cv2.putText(frame, 'red line', (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.line(frame, (8, 268), (927, 268), (255, 0, 0), 3)
    cv2.putText(frame, 'blue line', (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    downwards = len(set(counter_down))
    cv2.putText(frame, 'going down - ' + str(downwards), (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
    upwards = len(set(counter_up))
    cv2.putText(frame, 'going up - ' + str(upwards), (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    out.write(frame)  # Write the frame to the file
    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Exit on pressing 'ESC'
        break

cap.release()
out.release()
cv2.destroyAllWindows()

0: 320x640 2 persons, 17 cars, 2 motorcycles, 2 buss, 11 trucks, 1 umbrella, 15.0ms
Speed: 13.0ms preprocess, 15.0ms inference, 2.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 18 cars, 2 motorcycles, 2 buss, 11 trucks, 1 umbrella, 14.5ms
Speed: 2.7ms preprocess, 14.5ms inference, 2.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 18 cars, 2 motorcycles, 1 bus, 11 trucks, 11.0ms
Speed: 2.2ms preprocess, 11.0ms inference, 5.6ms postprocess per image at shape (1, 3, 320, 640)
0: 320x640 2 persons, 15 cars, 2 motorcycles, 2 buss, 12 trucks, 11.5ms
Speed: 3.0ms preprocess, 11.5ms inference, 3.7ms postprocess per image at shape (1, 3, 320, 640)
0: 320x640 2 persons, 16 cars, 2 motorcycles, 1 bus, 12 trucks, 11.5ms
Speed: 3.6ms preprocess, 11.5ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 640)
0: 320x640 2 persons, 15 cars, 2 motorcycles, 1 bus, 12 trucks, 1 umbrella, 13.9ms
Speed: 1.9ms preprocess, 13.9ms inference, 3.8m